In [1]:
import MDAnalysis as mda
from MDAnalysis.coordinates.memory import MemoryReader
from MDAnalysis.transformations import wrap 
from MDAnalysis.transformations.boxdimensions import set_dimensions
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import pickle as pkl

In [2]:
def pi_centrer(coords,weights,box_min, box_width):

    frac_coords = (coords - box_min) / box_width
    theta = frac_coords * (2 * np.pi) - np.pi
    xi = np.cos(theta)
    zeta = np.sin(theta)
    xi_bar = np.average(xi,weights=weights)
    zeta_bar = np.average(zeta,weights=weights)
    theta_bar = np.arctan2(zeta_bar, xi_bar)
    new_s_coords = (theta_bar + np.pi) / (2 * np.pi)
    new_s_coords = new_s_coords*box_width + box_min

    return new_s_coords

In [3]:
# masses = np.random.rand(5)*10   #np.array([10000,1,1,1,1])
# coords = np.random.rand(5,3)*20 
from tqdm import tqdm

particles = 5
box_min = 0
box_max = 20
box_width = box_max - box_min

broken = []
run_store = []

for i in tqdm(range(1000000)):


    masses = np.array((1,1,1,1,1))
    # masses[np.random.randint(0,particles)] *= 100



    x_coords = (np.random.uniform(low = -5, high = 5, size = particles)).astype('float32') % 20
    yz = np.zeros((2, particles))
    coords = np.concatenate((x_coords[np.newaxis, :], yz), axis=0).T


    u = mda.Universe.empty(5 ,trajectory = True)

    u.add_TopologyAttr('masses',masses)
    #u.add_TopologyAttr('dimensions', np.array[20,20,20,90,90,90])
    u.transfer_to_memory()
    #u.add_TopologyAttr('bonds', )



    reader = MemoryReader(coords)

    u.trajectory = reader
    dim = np.array([box_max, box_max, box_max, 90, 90, 90])
    transform1 = mda.transformations.boxdimensions.set_dimensions(dim)
    transform2 = wrap(u.atoms)
    workflow  = [transform1,transform2]
    u.trajectory.add_transformations(*workflow)

    u.add_bonds([tuple(range(i, i+2)) for i in range(0, 4)])

    u.atoms.unwrap()
    MDA_com = u.atoms.center_of_mass()[0]
    com_pi = pi_centrer(coords[:,0], masses,box_min,box_width)
    corrected_com = (np.average(((coords[:,0] - (com_pi + 0.5*box_width) ) % box_max), weights = masses) + (com_pi + 0.5*box_width)) % box_max
    wrapped = ((coords[:,0] - (com_pi + 0.5*box_width) ) % box_max)

    if np.isclose(MDA_com,corrected_com, atol = 1.1e-6):
        pass
    else:
        #raise ValueError
        broken.append([MDA_com,com_pi,corrected_com,coords,masses])
        
    run_store.append([MDA_com,com_pi,corrected_com,coords,masses])

100%|██████████| 1000000/1000000 [09:50<00:00, 1694.35it/s]


In [4]:
MDA_com,com_pi,corrected_com,coords,masses = zip(*run_store)

In [5]:
MDA_com = np.array(MDA_com)
com_pi = np.array(com_pi)
corrected_com = np.array(corrected_com)
coords= np.array(coords)
masses = np.array(masses)
pi_percent_diff = np.abs((np.abs(MDA_com - com_pi) / MDA_com) *100)
corr_percent_diff = np.abs((np.abs(MDA_com - corrected_com) / MDA_com) * 100)

In [6]:
df = pd.DataFrame(data={'MDA_com':MDA_com, 'com_pi':com_pi, 'corrected_com':corrected_com, 'pi_percent_diff':pi_percent_diff, 'corr_percent_diff':corr_percent_diff})

In [7]:
# df.to_pickle('1mill_com.pkl')
# np.save('masses.npy',masses)
# np.save('coords.npy',coords)

In [8]:
rms_p = np.sqrt(mean_squared_error(df['MDA_com'], df['com_pi']))
rms_cor = np.sqrt(mean_squared_error(df['MDA_com'], df['corrected_com']))

print(f"RMS PI: {rms_p}")
print(f"RMS CORR: {rms_cor}")

RMS PI: 3.782824332016914
RMS CORR: 0.020000000048558134


In [9]:
df.sort_values(by='pi_percent_diff', ascending=False).head(10)

,MDA_com,com_pi,corrected_com,pi_percent_diff,corr_percent_diff
282055,1.209974e-06,19.785179,9.357929e-07,1.635173e+09,2.266010e+01
867731,1.373291e-05,19.954875,1.376867e-05,1.453069e+08,2.604167e-01
349360,1.599789e-05,19.514644,1.573563e-05,1.219825e+08,1.639344e+00
748124,2.079010e-05,19.857598,2.048016e-05,9.551458e+07,1.490826e+00
218165,3.426075e-05,19.471453,3.376007e-05,5.683302e+07,1.461378e+00
816423,4.272461e-05,19.868368,4.312992e-05,4.650323e+07,9.486607e-01
761263,5.776882e-05,19.880306,5.726814e-05,3.441346e+07,8.666942e-01
185414,5.836487e-05,19.969610,5.860329e-05,3.421502e+07,4.084967e-01
700333,5.950928e-05,19.842598,5.955696e-05,3.334361e+07,8.012821e-02
517586,-7.629395e-07,0.230475,2.000000e+01,3.020889e+07,2.621440e+09


In [10]:
df.sort_values(by='corr_percent_diff', ascending=False).head(10)

,MDA_com,com_pi,corrected_com,pi_percent_diff,corr_percent_diff
517586,-7.629395e-07,0.230475,2.000000e+01,3.020889e+07,2.621440e+09
282055,1.209974e-06,19.785179,9.357929e-07,1.635173e+09,2.266010e+01
114847,3.900705e-06,0.003555,4.091440e-06,9.103339e+04,4.889754e+00
349360,1.599789e-05,19.514644,1.573563e-05,1.219825e+08,1.639344e+00
748124,2.079010e-05,19.857598,2.048016e-05,9.551458e+07,1.490826e+00
218165,3.426075e-05,19.471453,3.376007e-05,5.683302e+07,1.461378e+00
822420,2.288818e-05,0.153388,2.312064e-05,6.700641e+05,1.015625e+00
539876,2.940893e-05,0.151334,2.912283e-05,5.144860e+05,9.728415e-01
816423,4.272461e-05,19.868368,4.312992e-05,4.650323e+07,9.486607e-01
761263,5.776882e-05,19.880306,5.726814e-05,3.441346e+07,8.666942e-01


In [11]:
len(df)

1000000

In [12]:
df.loc[df['pi_percent_diff'] < df['corr_percent_diff']]

,MDA_com,com_pi,corrected_com,pi_percent_diff,corr_percent_diff
517586,-7.629395e-07,0.230475,19.999999,3.020889e+07,2.621440e+09


# Pi outperforming Corr

In [13]:
df.loc[df['pi_percent_diff'] < df['corr_percent_diff']].head(10)

,MDA_com,com_pi,corrected_com,pi_percent_diff,corr_percent_diff
517586,-7.629395e-07,0.230475,19.999999,3.020889e+07,2.621440e+09


# How many times pi wrapped the wrong way

In [14]:
len(df.loc[np.abs(df['com_pi']- df['MDA_com']) > 19 ])

36386

# How many times corrected wrapped the wrong way

In [15]:
len(df.loc[np.abs(df['corrected_com']- df['MDA_com']) > 19 ])

1

In [16]:
df_test = df.copy()

In [17]:
df_test = df.copy()
df_test.loc[np.abs(df_test['MDA_com'] - df_test['com_pi']) > 10, 'com_pi'] = df_test['com_pi'] -20
df_test.loc[np.abs(df_test['MDA_com'] - df_test['corr_percent_diff']) > 10, 'corr_percent_diff'] = df_test['corr_percent_diff'] -20
df_test['pi_percent_diff'] = (np.abs(df_test['MDA_com'] - df_test['com_pi']) / df_test['MDA_com']) *100
df_test['corr_percent_diff'] = (np.abs(df_test['MDA_com'] - df_test['corrected_com']) / df_test['MDA_com']) *100

In [18]:
df_test.sort_values(by='pi_percent_diff', ascending=False).tail(10)

,MDA_com,com_pi,corrected_com,pi_percent_diff,corr_percent_diff
112915,4.469072e+00,4.469072,4.469072,9.044176e-06,5.962166e-14
212914,1.895676e+01,18.956757,18.956756,8.583101e-06,3.380061e-07
242507,1.781406e+01,17.814055,17.814056,7.369010e-06,0.000000e+00
407115,1.864314e+01,18.643142,18.643141,5.756775e-06,1.182940e-06
940147,1.797884e+01,17.978837,17.978838,5.435063e-06,0.000000e+00
997273,1.854081e+01,18.540805,18.540806,4.095381e-06,0.000000e+00
619650,1.921725e+01,19.217249,19.217249,2.779857e-06,2.481298e-07
307246,1.835353e+01,18.353527,18.353527,1.618444e-06,0.000000e+00
478714,1.859223e+01,18.592230,18.592230,4.360748e-07,0.000000e+00
517586,-7.629395e-07,0.230475,19.999999,-3.020889e+07,-2.621440e+09


In [19]:
df_test.sort_values(by='corr_percent_diff', ascending=False).head(10)

,MDA_com,com_pi,corrected_com,pi_percent_diff,corr_percent_diff
282055,0.000001,-0.214821,9.357929e-07,1.775425e+07,22.660098
114847,0.000004,0.003555,4.091440e-06,9.103339e+04,4.889754
349360,0.000016,-0.485356,1.573563e-05,3.033977e+06,1.639344
748124,0.000021,-0.142402,2.048016e-05,6.850495e+05,1.490826
218165,0.000034,-0.528547,3.376007e-05,1.542818e+06,1.461378
822420,0.000023,0.153388,2.312064e-05,6.700641e+05,1.015625
539876,0.000029,0.151334,2.912283e-05,5.144860e+05,0.972842
816423,0.000043,-0.131632,4.312992e-05,3.081952e+05,0.948661
761263,0.000058,-0.119694,5.726814e-05,2.072953e+05,0.866694
905224,0.000028,0.432053,2.808571e-05,1.525287e+06,0.841751


In [20]:
rms_pi = np.sqrt(mean_squared_error(df_test['MDA_com'], df_test['com_pi']))
rms_corr = np.sqrt(mean_squared_error(df_test['MDA_com'], df_test['corrected_com']))

print(f"RMS PI: {rms_pi}")
print(f"RMS CORR: {rms_corr}")

RMS PI: 5.394683461236905
RMS CORR: 0.020000000048558134


In [21]:
df_stand = df.copy()
df_stand = df_stand.sort_values(by='pi_percent_diff', ascending=False)
df_stand = df_stand[30000:]
df_stand.head(5)

,MDA_com,com_pi,corrected_com,pi_percent_diff,corr_percent_diff
845784,0.153594,0.513197,0.153595,234.124783,9.313565e-05
900018,0.304724,1.018116,0.304723,234.111438,1.095373e-04
458464,0.028066,0.093768,0.028066,234.104738,4.672278e-04
974057,0.105326,0.351876,0.105326,234.082228,2.701085e-12
650913,0.256656,0.857375,0.256656,234.055893,9.289414e-05


In [22]:
df_stand['corr_percent_diff'].mean()

1.5339107452872067e-05

In [23]:
df_stand['pi_percent_diff'].mean()

15.875702974241989

In [24]:
np.arctan2(0,0)

0.0

In [25]:
len(MDA_com)

1000000